In [1]:
from src.models.modelling_llama_skip import LlamaSkipConnectionModel, LlamaSkipConnectionConfig

In [5]:
checkpoint = "meta-llama/Llama-3.2-1B"
llamaSkipConfig = LlamaSkipConnectionConfig.from_json_file("./configs/llama_skip_config.json")
llamaSkipModel = LlamaSkipConnectionModel.from_pretrained(checkpoint, config=llamaSkipConfig)